`
!python -m wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv -o data.csv
`    

### Model creation

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [7]:
df = pd.read_csv("data.csv")

In [6]:
df['target'] = df['card'].apply(lambda x: 1 if x == "yes" else 0).astype(int)

In [7]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.2, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.target.values
y_val = df_val.target.values
y_test = df_test.target.values

del df_train['target']
del df_val['target']
del df_test['target']

In [8]:
numerical = ["reports", "age", "income", "share", "expenditure", "dependents", "months", "active"]
categorical = ["owner" ,"selfemp" ,"majorcards"]

In [9]:
cols_to_keep = ["reports", "age", "income", 
                "share", "expenditure", "dependents", 
                "months", "majorcards", "active", 
                "owner", "selfemp"]

df_train = df_train[cols_to_keep]
df_val = df_val[cols_to_keep]
df_test = df_test[cols_to_keep]

In [10]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train, y_train)
    
    return dv, model



def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')

    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [11]:
C = 1.0
n_splits = 5

In [12]:
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

scores = []

for train_idx, val_idx in kfold.split(df_full_train):
    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.target.values
    y_val = df_val.target.values

    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print('C=%s %.3f +- %.3f' % (C, np.mean(scores), np.std(scores)))

C=1.0 0.996 +- 0.003


In [13]:
scores

[0.9943244739756367,
 0.9944852941176471,
 0.9938076416337287,
 0.9997405293201868,
 1.0]

In [15]:
dv, model = train(df_full_train, df_full_train.target.values, C=1.0)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
auc

0.9960208142026324

***

&nbsp;

### Save the model

In [16]:
import pickle

In [17]:
output_file = f'model_C={C}.bin'

In [18]:
output_file

'model_C=1.0.bin'

In [19]:
f_out = open(output_file, 'wb') 
pickle.dump((dv, model), f_out)
f_out.close()

In [22]:
with open(output_file, 'wb') as f_out: 
    pickle.dump((dv, model), f_out)

***

&nbsp;

### Load model

In [1]:
import pickle

In [2]:
input_file = 'model_C=1.0.bin'

In [3]:
with open(input_file, 'rb') as f_in: 
    dv, model = pickle.load(f_in)

In [4]:
model

LogisticRegression(max_iter=1000)

In [11]:
# Set of features from which we will predict an output

customer = {
    
    'card': 'yes', 
    'reports': 0, 
    'age': 35, 
    'income': 3.12, 
    'share': 0.004318, 
    'expenditure': 21.86, 
    'owner': 'yes',
    'selfemp': 'no', 
    'dependents': 2, 
    'months': 38, 
    'majorcards': 1, 
    'active': 10
}

In [12]:
X = dv.transform([customer])

In [13]:
y_pred = model.predict_proba(X)[0, 1]

In [14]:
print('input:', customer)
print('output:', y_pred)

input: {'card': 'yes', 'reports': 0, 'age': 35, 'income': 3.12, 'share': 0.004318, 'expenditure': 21.86, 'owner': 'yes', 'selfemp': 'no', 'dependents': 2, 'months': 38, 'majorcards': 1, 'active': 10}
output: 0.9999999999999989
